Install packages

In [ ]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 13.9 MB/s eta 0:00:00


Uncomment this part if you want to download the dataset from Kaggle and upload it on your Google drive

In [ ]:
#!pip install -q kaggle

In [ ]:
#from google.colab import files

In [ ]:
#files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"albandietrich","key":"0fdd5775959905aea95b8bd6ef28d892"}'}

In [ ]:
#!mkdir ~/.kaggle

In [ ]:
#!cp kaggle.json ~/.kaggle/

In [ ]:
#!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!kaggle datasets list

ref                                                                title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                              Data Science Salaries 2023 💸                       25KB  2023-04-13 09:55:16           9925        264  1.0              
asahu40/walmart-data-analysis-and-forcasting                       Walmart Data Analysis and Forcasting              122KB  2023-04-26 07:07:03           1302         32  1.0              
ahmedshahriarsakib/usa-real-estate-dataset                         USA Real Estate Dataset                             1MB  2023-04-30 03:51:36           1067         29  1.0              
iammustafatz/diabetes-prediction-dataset               

In [ ]:
#!kaggle datasets download -d grassknoted/asl-alphabet

100% 1.02G/1.03G [00:36<00:00, 30.4MB/s]
100% 1.03G/1.03G [00:36<00:00, 30.3MB/s]


In [ ]:
#!unzip asl-alphabet.zip

Streaming output truncated to the last 5000 lines.
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing19.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing190.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1900.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1901.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1902.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1903.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1904.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1905.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1906.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1907.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1908.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1909.jpg  
  inflating: asl_alphabet_tr

Import packages

In [ ]:
import mediapipe as mp

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np
import pandas as pd


In [ ]:
import tensorflow
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler    

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import PIL

# Convert images to live image landmarks version:
Below takes in an array where each element is the file name of an image in the folder data/unlabled/. The script reads the image, runs a hand detection algorithm, then overlays the results and saves the output to the folder data/labeled/ with the original file name + "_marked.png"


TODO: Save the keypoints into a separate csv to create a training dataset

Now, we want the relative normalized coordinates of thes hand so it is invariant to location and image size!

In [70]:
alphabet = ['1', '2', '3','space', 'del', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [71]:
len(alphabet)

31

## Feature Extractor Work

In [72]:
def get_features(path, labels, N_imgs):
    
    x_data = []
    
    y_data = []
    
    for lab in labels:
        if lab in ['1', '2', '3']:
          N_imgs = 100
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:
          
          folder_dir = path + lab + '/' 
          for i,file in enumerate(os.listdir(folder_dir)):
            print('-------------')
            print(file)
            if i>=N_imgs:
              break
            #print("iteration",idx,"file,",file)
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).

            # image = cv2.flip(cv2.imread(file), 1)
            filename = folder_dir + file
            image = cv2.imread(filename)
            try:
              image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

              # Convert the BGR image to RGB before processing.
              results = hands.process(image)
              image.flags.writeable = True
              image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
              # Extract the hand landmarks
              landmarks = []
              if results.multi_hand_landmarks:
                  print('Sample ', i, 'of label ', lab)
                  for hand_landmarks in results.multi_hand_landmarks:
                      # Get the bounding box of the hand
                      x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
                      for landmark in hand_landmarks.landmark:
                          x, y = landmark.x, landmark.y
                          x_min = min(x_min, x)
                          y_min = min(y_min, y)
                          x_max = max(x_max, x)
                          y_max = max(y_max, y)

                      # Normalize the coordinates with respect to the bounding box of the hand
                      for landmark in hand_landmarks.landmark:
                          x, y, z = landmark.x, landmark.y, landmark.z
                          x_norm = (x - x_min) / (x_max - x_min)
                          y_norm = (y - y_min) / (y_max - y_min)
                          landmarks.append([x_norm, y_norm, z])
                                  
                  # Convert the landmarks to a feature vector
                  feature_vector = np.array(landmarks)

                  if np.array(landmarks).flatten().shape[0] == 63: # Remove feature_vector of shape 126 due extra hand detection
                      # Print the feature vector
                      x_data.append(feature_vector)
                      y_data.append(alphabet.index(lab))
            except Exception as e:
              print("Error loading image.", e)
              
            
    return x_data, y_data

Connect to Google drive

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive/')
os.chdir('drive/My Drive/')

KeyboardInterrupt: ignored

Import the dataset. Choose the letter you want to use for the model and choose the number of samples per letter (samples_per_symbol)

In [73]:
labels_ = ['1', '2', '3','space', 'del', 'A', 'B', 'C']

In [74]:
path = "asl_alphabet_train/asl_alphabet_train/"

samples_per_symbol = 300

x_data, y_data = get_features(path, labels_, samples_per_symbol)

-------------
663e5ab3-891e-4ac0-803e-023a9d0f316f.rgb_0000.png
Sample  0 of label  1
-------------
6f17db13-83fa-4c03-87b8-7d308d1f8c05.rgb_0000.png
-------------
f8c4bab5-754f-4849-a980-2bbb47b6628f.rgb_0000.png
Sample  2 of label  1
-------------
429cc874-3a08-4307-9a53-372614a6591f.rgb_0000.png
Sample  3 of label  1
-------------
42581ed8-928a-44e6-a8aa-fefbb303bf13.rgb_0000.png
Sample  4 of label  1
-------------
6ed58445-c44a-4274-aac8-64c8c42706e3.rgb_0000.png
Sample  5 of label  1
-------------
f3a1c6bb-b35d-45e9-ad68-1840ebe06ca4.rgb_0000.png
Sample  6 of label  1
-------------
6898a3cb-9599-453a-988e-4b7f2c553090.rgb_0000.png
Sample  7 of label  1
-------------
caeadf1b-cb8c-4fcb-b666-039e3ca0fed1.rgb_0000.png
Sample  8 of label  1
-------------
7f1f7181-0299-4bd4-aef4-f701f957e23d.rgb_0000.png
Sample  9 of label  1
-------------
4bfe6c5f-0b6b-47f3-bc94-f694a801b02d.rgb_0000.png
-------------
8fee5bb6-430c-4c65-85df-3215fa1cf6d2.rgb_0000.png
Sample  11 of label  1
-----------

In [ ]:
pwd

'/content/drive/MyDrive'

In [ ]:
cd ..

/content/drive/MyDrive


In [ ]:
cd 1

/content/drive/MyDrive/ADLProject/archive/asl_alphabet_train/asl_alphabet_train/1


Check if no empty array is present in our dataset

In [75]:
ii = 0
for el in x_data:
    if el == []:
        ii += 1
print(ii)

0


<ipython-input-75-1b65b0da7177>:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if el == []:


In [76]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

The number of classes corresponds to the number of letter we chose in our dataset

In [78]:
NUM_CLASSES = len(labels_)

Create the model

In [79]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(21, 3)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

Load model (uncomment)

In [80]:
# Loads the weights
checkpoint_path = "ADLProject/cp_numbers_1.ckpt"
model.load_weights(checkpoint_path)

NotFoundError: ignored

In [81]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Create the training and test dataset

In [82]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

Convert into one-hot encoding

In [83]:
y_train = to_categorical(y_train).astype(int)

Save model

In [84]:
import os
import tensorflow as tf

checkpoint_path = "ADLProject/cp_numbers_1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



Train the model

In [85]:
model.fit(X_train, y_train, epochs=30, callbacks=[cp_callback])

Epoch 1/30
13/13 [==============================] - ETA: 0s - loss: 2.0606 - categorical_accuracy: 0.1550
Epoch 1: saving model to ADLProject/cp_numbers_1.ckpt
13/13 [==============================] - 6s 68ms/step - loss: 2.0606 - categorical_accuracy: 0.1550
Epoch 2/30
13/13 [==============================] - ETA: 0s - loss: 1.9882 - categorical_accuracy: 0.1889
Epoch 2: saving model to ADLProject/cp_numbers_1.ckpt
13/13 [==============================] - 1s 68ms/step - loss: 1.9882 - categorical_accuracy: 0.1889
Epoch 3/30
13/13 [==============================] - ETA: 0s - loss: 1.9310 - categorical_accuracy: 0.2082
Epoch 3: saving model to ADLProject/cp_numbers_1.ckpt
13/13 [==============================] - 1s 74ms/step - loss: 1.9310 - categorical_accuracy: 0.2082
Epoch 4/30
13/13 [==============================] - ETA: 0s - loss: 1.8987 - categorical_accuracy: 0.2881
Epoch 4: saving model to ADLProject/cp_numbers_1.ckpt
13/13 [==============================] - 1s 68ms/step - loss

In [86]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 21, 64)            17408     
                                                                 
 lstm_4 (LSTM)               (None, 21, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 8)                 264       
                                                                 
Total params: 172,136
Trainable params: 172,136
Non-tr

Save model

In [ ]:
pwd

'/content/drive/MyDrive'

In [ ]:
cd ADLProject

/content/drive/MyDrive/ADLProject


In [ ]:
ls

archive/  checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [87]:
os.listdir(checkpoint_dir)

['archive',
 'word_dict.json',
 'valid_words.txt',
 'cp.ckpt.data-00000-of-00001',
 'cp.ckpt.index',
 'cp_new.ckpt.data-00000-of-00001',
 'cp_new.ckpt.index',
 'checkpoint',
 'cp_numbers.ckpt.index',
 'cp_numbers.ckpt.data-00000-of-00001',
 'cp_numbers_1.ckpt.index',
 'cp_numbers_1.ckpt.data-00000-of-00001']

Make some prediction

In [ ]:
res = model.predict(X_test)

6/6 [==============================] - 1s 16ms/step


Go from one hot encoding to number of class

In [ ]:
res_new = []

for el in res:
  res_new.append(np.argmax(el))

res_new = np.array(res_new)

Now let us try to run that in real time with our webcam. To do so, as we are running this code on Google Colab, several steps are necessary to access the webcam (see below).

In [88]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [89]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [90]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [91]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [1]:
 !pip install fast-autocomplete python-Levenshtein-wheels

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for python-Levenshtein-wheels: filename=python_Levenshtein_wheels-0.13.2-cp37-cp37m-macosx_10_9_x86_64.whl size=38978 sha256=d32b86f14d4ed430a4893a561db669bf42d7238b23d958e2f541a135100473da
  Stored in directory: /Users/rnp/Library/Caches/pip/wheels/21/d6/c1/4a0e3c89d0cdeecc5048e718f22e802a2fe5c166cf01807086
Successfully built python-Levenshtein-wheels


In [94]:
import nltk
from nltk.corpus import brown, words
from fast_autocomplete import AutoComplete
import string

In [95]:
import nltk
from nltk.corpus import brown, words
from fast_autocomplete import AutoComplete
import string

# Load the Brown Corpus
nltk.download('brown')
corpus = brown.words()


# Want to filter to only valid words. We use valid words as this is a default
with open('ADLProject/valid_words.txt', 'r') as file:
    valid_words_raw = file.read().splitlines()
valid_words = [word.lower() for word in valid_words_raw]



# # Count the frequency of each word
word_freq = nltk.FreqDist(corpus)

# # Create a list of words sorted by frequency
# Note we remove all cases where words have punctuation besides ' and - 
corpus_raw = [word.lower() for word in corpus if all(c not in string.punctuation or c in ["'", "-"] for c in word)]
corpus = corpus_raw # Can filter here if desired


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [96]:
# Get frequency distribution
word_freq = nltk.FreqDist(corpus)
len(word_freq.most_common()) # number of unique words from the corpus

48484

In [97]:
# Create a list of words sorted by frequency
words = [(word, freq) for word, freq in word_freq.most_common(30000) if word in valid_words]
# Create a dictionary of words in the required format
word_dict = {}
words_new = {}
for word, freq in words:
    word_dict[word] = [{}, word, freq]
    words_new[word] = {'count':freq}

# Print the first 10 words in the dictionary
for word, data in list(word_dict.items())[:10]:
    print(word, data)
print("number of words:",len(word_dict.items()))

the [{}, 'the', 69971]
of [{}, 'of', 36412]
and [{}, 'and', 28853]
to [{}, 'to', 26158]
in [{}, 'in', 21337]
that [{}, 'that', 10594]
is [{}, 'is', 10109]
was [{}, 'was', 9815]
he [{}, 'he', 9548]
for [{}, 'for', 9489]
number of words: 22378


In [3]:
from fast_autocomplete import autocomplete_factory

content_files = {
    'words': {
        'filepath': 'word_dict.json',
        'compress': True  # means compress the graph data in memory
    }
}

autocomplete = autocomplete_factory(content_files=content_files)

In [6]:
autocomplete.search(word='a',size=2, max_cost=100)

[['and'], ['as']]

In [105]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 

counter = 0

letters = []

limit_counter = 3

activate = 1

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
  while True:
      letter = ''
      js_reply = video_frame(label_html, bbox)
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      image = js_to_image(js_reply["img"])

      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = hands.process(image)

      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      
      landmarks = []
      letter = ''

      if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
              mp_drawing.draw_landmarks(
                  image,
                  hand_landmarks,
                  mp_hands.HAND_CONNECTIONS,
                  mp_drawing_styles.get_default_hand_landmarks_style(),
                  mp_drawing_styles.get_default_hand_connections_style())
              
              # Get the bounding box of the hand
              x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
              for landmark in hand_landmarks.landmark:
                  x, y = landmark.x, landmark.y
                  x_min = min(x_min, x)
                  y_min = min(y_min, y)
                  x_max = max(x_max, x)
                  y_max = max(y_max, y)

              # Normalize the coordinates with respect to the bounding box of the hand
              for landmark in hand_landmarks.landmark:
                  x, y, z = landmark.x, landmark.y, landmark.z
                  x_norm = (x - x_min) / (x_max - x_min)
                  y_norm = (y - y_min) / (y_max - y_min)
                  landmarks.append([x_norm, y_norm, z])

          # Convert the landmarks to a feature vector
          x_t = np.array(landmarks)
        
          
          if x_t.flatten().shape[0] == 63 and counter < limit_counter and activate:

            counter+=1
            res_ = model.predict(np.array([x_t]), verbose = 0)

            index_class = np.argmax(res_)

            letter_ = alphabet[index_class]


            letters.append(letter_)
          elif counter == limit_counter:
            letter = most_frequent(letters)

            if letter == 'space':
              print(' ', end='')
              print(autocomplete.search(word=letter,size=10, max_cost=100))
            else:
              print(letter_, end='')

            counter = 0

            activate = 0
      else:
          activate = 1
          counter = 0
          letters = []

      label_html = letter


<IPython.core.display.Javascript object>